# Data Harvesting & Registration

In [ ]:
domain = "185-52-195-215.nip.io"

### S2L2A 1 day of data harvesting

Ad-hoc harvesting can be invoked by provision of a suitable config.yaml into the harvester pod, which can then be invoked for the default harvester configuration.

```yaml
harvesters:
- filter: {}
  name: Creodias-Opensearch
  postprocess:
  - type: harvester_eoepca.postprocess.CREODIASOpenSearchSentinel2Postprocessor
  queue: register
  resource:
    format_config:
      property_mapping:
        end_datetime: completionDate
        productIdentifier: productIdentifier
        start_datetime: startDate
      type: application/json
    query:
      bbox: 14.9,47.7,16.4,48.7
      collection: null
      time:
        begin: "2019-09-10T00:00:00Z"
        end: "2019-09-11T00:00:00Z"
        property: sensed
    type: OpenSearch
    url: https://finder.creodias.eu/resto/api/collections/Sentinel2/describe.xml
redis:
  host: data-access-redis-master
  port: 6379
```

In [ ]:
# trigger harvest with a default collection
!k -n rm exec -it deployment/data-access-harvester -- bash -c 'cat /config.yaml'
!k -n rm exec -it deployment/data-access-harvester -- bash -c 'python3 -m harvester harvest --config-file /config.yaml --host data-access-redis-master --port 6379 Creodias-Opensearch'

In [ ]:
# check status of registration 
!k -n rm logs deployment/data-access-registrar


In [ ]:
# check success and exceptions of registration 
!k -n rm logs deployment/data-access-registrar | grep Successful
!k -n rm logs deployment/data-access-registrar | grep Exception

In [ ]:
# check existence of new products in the EOxServer database by listing all products

k -n rm exec -it deployment/data-access-registrar -- bash -c 'python3 $INSTANCE_DIR/manage.py id list -t Product -s '

In [ ]:
# or get their count directly at the EOxServer OpenSearch Endpoint
import requests
base = f"data-access.{domain}"
S2L2A_opensearch_url = f"https://{base}/opensearch/collections/S2L2A/atom/?count=0"
r = requests.get(url = S2L2A_opensearch_url).text
print(r)

# Harvesting configuration

- show pycsw resource catalog
- ask participants to change bbox to their area of origin
- add a link to the schema of harvester

# Collection/ProductType values